<a href="https://colab.research.google.com/github/anaghasid/GenPruning/blob/main/T5_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet datasets
!pip install --quiet evaluate
!pip install --quiet accelerate
!pip install --quiet bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from datasets import load_dataset
import accelerate
from sklearn.metrics import accuracy_score
import time
from torch.utils.data import DataLoader
from collections import defaultdict
import pandas as pd
import random

In [3]:
def make_model():
  model_name = "google-t5/t5-small"
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True,
                                                           device_map=device)
  return model

In [4]:
# Load a pretrained model and tokenizer
model_name = "google-t5/t5-small"
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)
model = make_model()
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [5]:
print("Number of encoders/decoders:", model.config.num_layers)
print("Number of heads:", model.config.num_heads)

Number of encoders/decoders: 6
Number of heads: 8


In [6]:
# Load the GLUE dataset (SST-2 as an example)
dataset = load_dataset("glue", "cola", split='validation', trust_remote_code=True)
print("Sample from the dataset:", dataset[0])
metric = evaluate.load("glue", "cola", trust_remote_code=True)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Sample from the dataset: {'sentence': 'The sailors rode the breeze clear of the rocks.', 'label': 1, 'idx': 0}


In [7]:
print(len(dataset))
dataset[0:10]

1043


{'sentence': ['The sailors rode the breeze clear of the rocks.',
  'The weights made the rope stretch over the pulley.',
  'The mechanical doll wriggled itself loose.',
  'If you had eaten more, you would want less.',
  'As you eat the most, you want the least.',
  'The more you would want, the less you would eat.',
  'I demand that the more John eat, the more he pays.',
  'Mary listens to the Grateful Dead, she gets depressed.',
  'The angrier Mary got, the more she looked at pictures.',
  'The higher the stakes, the lower his expectations are.'],
 'label': [1, 1, 1, 1, 0, 0, 0, 1, 1, 1],
 'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [8]:
# Preprocess the data
def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=128)
    tokenized_inputs['label'] = examples['label']
    return tokenized_inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

In [9]:
# Use DataLoader to create batches
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
dataloader = DataLoader(encoded_dataset, batch_size=16)

In [10]:
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

dataloader = DataLoader(encoded_dataset, batch_size=16)


# Display a sample batch from the dataloader
for batch in dataloader:
    # print(batch['label'])
    input_ids = batch['input_ids'].tolist()
    attention_masks = batch['attention_mask'].tolist()
    labels = batch['label'].tolist()

    truncated_input_ids = [ids[:10] for ids in input_ids]  # Show only the first 10 tokens
    truncated_attention_masks = [masks[:10] for masks in attention_masks]

    df = pd.DataFrame({
        'Input IDs': truncated_input_ids,
        'Attention Mask': truncated_attention_masks,
        'Label': labels
    })

    print(df)
    break


                                                Input IDs                  Attention Mask  Label
0      [37, 30899, 6102, 15, 8, 15825, 964, 13, 8, 12288]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      1
1        [37, 1293, 7, 263, 8, 13888, 6606, 147, 8, 3197]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      1
2   [37, 8168, 14295, 3, 210, 23983, 1361, 1402, 6044, 5]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      1
3         [156, 25, 141, 16929, 72, 6, 25, 133, 241, 705]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      1
4               [282, 25, 3, 1544, 8, 167, 6, 25, 241, 8]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      0
5              [37, 72, 25, 133, 241, 6, 8, 705, 25, 133]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      0
6              [27, 2173, 24, 8, 72, 1079, 3, 1544, 6, 8]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      0
7        [3790, 3011, 7, 12, 8, 350, 2206, 1329, 9651, 6]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      1
8           [37, 3, 1468, 6711, 3790, 530, 6, 8, 72, 255]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      1
9           [37, 1146, 8, 8474

In [11]:
# Modify model based on chromosome
def modify_model(model, chromosome):
    num_heads = model.config.num_heads
    num_blocks = model.config.num_layers
    print(num_blocks)
    # num_ffn = model.config.num_hidden_layers

    # Disable attention heads
    heads_to_prune = defaultdict(list)
    for i, gene in enumerate(chromosome):
        if gene == 0:
          block_num = i//num_heads
          head_num = i%num_heads
          heads_to_prune[block_num].append(head_num)

    print(heads_to_prune)
    if heads_to_prune:

        for block_num in heads_to_prune:
            print("Pruning")
            model.transformer.encoder.block[block_num].layer[0].SelfAttention.prune_heads(heads_to_prune[block_num])
            # if(bloack_num!=0):
            model.transformer.decoder.block[block_num].layer[0].SelfAttention.prune_heads(heads_to_prune[block_num])
            # model.transformer.decoder.block[block_num].layer[1].EncDecAttention.prune_heads(heads_to_prune[block_num])

            # print(block_num, model.transformer.encoder.block[block_num].layer[0].SelfAttention.n_heads)
            # model.prune_heads(heads_to_prune)

            # model.transformer.encoder.block[block_num].layer[0].SelfAttention.q = prune_linear_layer(model.transformer.encoder.block[block_num].layer[0].SelfAttention.q, index)
            # model.transformer.encoder.block[block_num].layer[0].SelfAttention.k = prune_linear_layer(model.transformer.encoder.block[block_num].layer[0].SelfAttention.k, index)
            # model.transformer.encoder.block[block_num].layer[0].SelfAttention.v = prune_linear_layer(model.transformer.encoder.block[block_num].layer[0].SelfAttention.v, index)
            # model.transformer.encoder.block[block_num].layer[0].SelfAttention.o = prune_linear_layer(model.transformer.encoder.block[block_num].layer[0].SelfAttention.o, index, dim=1)

In [12]:
def evaluate_fitness(chromosome, model, encoded_dataset, metric):
    # Modify the model according to the chromosome
    if len(chromosome):
      model = make_model()
      modify_model(model, chromosome)

    start_time = time.time()
    predictions = []
    labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            outputs = model(input_ids, attention_mask=attention_mask)

            # Get predictions
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            predictions.extend(preds.cpu().numpy())
            labels.extend(batch['label'].cpu().numpy())

    # Print a few predictions and their corresponding labels
    # for i in range(5):
    #     print(f"Prediction: {predictions[i]}, Label: {labels[i]}")

    # Evaluate the model
    print("For the chromosome", chromosome)
    fitness = accuracy_score(labels, predictions)
    print(f"Validation Accuracy: {fitness:.4f}")
    end_time = time.time()
    print("Execution time = ",end_time-start_time)
    print()

    return fitness

In [13]:
# without modifying the model
print(evaluate_fitness([], model, encoded_dataset, metric))

For the chromosome []
Validation Accuracy: 0.6913
Execution time =  240.3186240196228

0.6912751677852349


In [14]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [15]:
def create_random_binary_list(length, percentage_of_zeros):
    # Calculate the number of zeros and ones based on the percentage
    num_zeros = int(length * (percentage_of_zeros / 100))
    num_ones = length - num_zeros

    # Create the list with the required number of 0s and 1s
    binary_list = [0] * num_zeros + [1] * num_ones

    # Shuffle the list to randomize the order
    random.shuffle(binary_list)

    return binary_list

def initialize_chromosome(num_genes):
  # initialize chromosome with given sparsity percentage
  return create_random_binary_list(num_genes, 30)

In [16]:
# checking to see if decoder is there
print(model.config.is_decoder)

False


In [17]:
model  = make_model()
chromosome = initialize_chromosome(model.config.num_heads*model.config.num_layers)
print("Chromosome:", chromosome)
fitness = evaluate_fitness(chromosome, model, encoded_dataset, metric)
print("Fitness score (accuracy):", fitness)


Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Chromosome: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1]


Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
defaultdict(<class 'list'>, {1: [0, 1, 2, 3], 2: [0, 1, 4], 3: [5, 7], 4: [0, 4, 7], 5: [0, 6]})
Pruning
Pruning
Pruning
Pruning
Pruning


KeyboardInterrupt: 

In [18]:
def find_size(model):
  total_size_in_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
  total_size_in_megabytes = total_size_in_bytes / (1024 ** 2)
  print(f"Model size: {total_size_in_megabytes:.2f} MB")

In [19]:
# m  = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True, device_map=device)
# print(find_size(m), find_size(model))

In [20]:
POPN_SIZE = 20
crossover_rate = 0.7
mutation_rate = 0.1

In [21]:
def initialize_population(chromosome_length):
  # initialize random population
  population = []
  for _ in range(POPN_SIZE):
    chromosome = initialize_chromosome(chromosome_length)
    population.append(chromosome)
  return population

In [22]:
# Selection (Tournament Selection)
def select_parents(population, fitness_scores):
    parents = []
    for _ in range(POPN_SIZE):
        tournament = np.random.choice(POPN_SIZE, 2)
        print("tournament = ",tournament)
        winner = tournament[np.argmax(fitness_scores[tournament])]
        parents.append(population[winner])
    return np.array(parents)

In [23]:
# Crossover (Single-point crossover)
def crossover(parent1, parent2):
    if np.random.rand() < crossover_rate:
        point = np.random.randint(1, len(parent1) - 1)
        child1 = np.concatenate([parent1[:point], parent2[point:]])
        child2 = np.concatenate([parent2[:point], parent1[point:]])
    else:
        child1, child2 = parent1, parent2
    return child1, child2

In [24]:
# Mutation (Flip bit mutation)
def mutate(chromosome):
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

In [31]:
def genetic_algorithm(model, num_generations, desired_sparsity):
  population = initialize_population(model.config.num_heads*model.config.num_layers)
  for generation in range(num_generations):
      fitness_scores = np.array([evaluate_fitness(chrom, model, encoded_dataset, metric) for chrom in population])
      parents = select_parents(population, fitness_scores)
      new_population = []
      for i in range(0, POPN_SIZE, 2):
          parent1, parent2 = parents[i], parents[i + 1]
          child1, child2 = crossover(parent1, parent2)
          child1 = mutate(child1)
          child2 = mutate(child2)
          new_population.extend([child1, child2])
      population = np.array(new_population)

      # Check for desired sparsity level
      sparsity_levels = np.mean(population == 0, axis=1)
      print(sparsity_levels)
      if np.any(sparsity_levels >= desired_sparsity):
          best_chromosome = population[np.argmax(sparsity_levels)]
          break
  return best_chromosome

In [32]:
genetic_algorithm(model, 10, 0.3)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
defaultdict(<class 'list'>, {0: [0, 1, 4], 1: [2, 5, 6, 7], 2: [1], 3: [4, 6], 4: [1, 7], 5: [1, 7]})
Pruning
Pruning
Pruning
Pruning
Pruning
Pruning
For the chromosome [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0]
Validation Accuracy: 0.3538
Execution time =  220.13971972465515



Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
defaultdict(<class 'list'>, {0: [5], 1: [3, 4, 5, 6], 2: [4, 7], 4: [2, 3, 4, 5], 5: [1, 5, 7]})
Pruning
Pruning
Pruning
Pruning
Pruning
For the chromosome [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0]
Validation Accuracy: 0.3730
Execution time =  222.45740389823914



Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
defaultdict(<class 'list'>, {0: [5, 6], 1: [3, 4], 2: [3, 5, 6], 3: [0, 5], 4: [1, 6, 7], 5: [3, 7]})
Pruning
Pruning
Pruning
Pruning
Pruning
Pruning
For the chromosome [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0]
Validation Accuracy: 0.3087
Execution time =  211.41342115402222



Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
defaultdict(<class 'list'>, {0: [1, 7], 1: [7], 2: [2, 3, 7], 3: [1, 3, 5, 6], 4: [4], 5: [2, 3, 6]})
Pruning
Pruning
Pruning
Pruning
Pruning
Pruning
For the chromosome [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1]
Validation Accuracy: 0.6913
Execution time =  212.1215159893036

tournament =  [5 5]
tournament =  [ 1 19]
tournament =  [13 19]
tournament =  [2 6]
tournament =  [8 7]
tournament =  [18 11]
tournament =  [16 16]
tournament =  [12  4]
tournament =  [10 15]
tournament =  [3 1]
tournament =  [2 5]
tournament =  [6 8]
tournament =  [14 14]
tournament =  [6 5]
tournament =  [ 2 15]
tournament =  [ 4 17]
tournament =  [15  7]
tournament =  [12 16]
tournament =  [15  5]
tournament =  [ 3 12]
[0.35416667 0.3125     0.33333333 0.22916667 0.33333333 0.41666667
 0.3125     0.39583333 0.25       0.35416667 0.3125     0.33333333
 0.35416667 0.33333333 0.375      0.41666667 0.29166667 0.2

array([0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1])